# Test model example 

This notebook tests the different models (HAY, HAY_AIS, HAY_AIS_HILLOCK) on a simple step stimulus.

In [ ]:
import json
import sys
import os
import time
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import shutil

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
probe_type = "planar"
model_name = "hay_ais_hillock" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = Path("..") / "cell_models"

model_folder = cell_models_folder / f"{model_name}_model"

In [ ]:
# instantiate cell model
cell = mf.create_ground_truth_model(model_name=model_name, 
                                    cell_model_folder=model_folder, 
                                    release=True, v_init=-80)

# define a probe (this can also be defined externally)
probe = mf.define_electrode(probe_type=probe_type)

# define a test protocol
protocols = mf.define_test_step_protocol(step_amplitude=0.5, tot_duration=2000, step_duration=1500, probe=probe)

# instantiate simulator
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

In [ ]:
t_start = time.time()
responses = protocols["TestStep"].run(cell, param_values={}, sim=sim)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
mf.plot_responses(responses)

In [ ]:
eap = mf.calculate_eap(responses, protocols=protocols, protocol_name="TestStep")

In [ ]:
ax = mf.plot_eap(responses, protocols=protocols, protocol_name="TestStep", probe=probe)

## (optional) save EAP for comparison among models

In [ ]:
eap_folder = Path("eap")
eap_folder.mkdir(exist_ok=True)

np.save(eap_folder / f"eap_{model_name}.npy", eap)

